In [ ]:
import pandas as pd
import numpy as np
import keras as keras

# Estos tienen solo una sola salida. 24x24
df_letras_train = pd.read_csv("DatasetsIAO/Letras/sign_mnist_train.csv")
df_letras_test = pd.read_csv("DatasetsIAO/Letras/sign_mnist_test.csv")

# Estos están en one hot encoding. 64x64
X_numeros = np.load("DatasetsIAO/NumerosBien/X.npy")
Y_numeros = np.load("DatasetsIAO/NumerosBien/Y.npy")

1. Cambiar Y numeros a solo tener una salida **DONE**
2. Juntar test y train de letras y separar en Y y X **DONE**
3. Reescalar las imagenes de numeros de 64x64 a 28x28 **DONE**
4. Juntar los datos de numeros y los de letras **DONE**
5. Pasar todo el test a one hot encoding
6. Separar en train y test.

In [ ]:
Y_numeros.shape

### Transformamos a dataframe el np.array de Numeros

In [ ]:
# Para manejar los dataset de los números, los pasamos a dataframe

# Hacemos el reshape de X_numeros para que solo tenga dos dimensiones
# y pueda ser representado en un dataframe.
X_numeros = X_numeros.reshape(2062, 4096)

# Pasamos tanto Y_numeros como X_numeros a dataframe.
X_numeros = pd.DataFrame(data = X_numeros,
          index=np.arange(1, X_numeros.shape[0] + 1),
          columns=np.arange(1, X_numeros.shape[1] + 1))

Y_numeros = pd.DataFrame(data = Y_numeros,
          index=np.arange(1, Y_numeros.shape[0] + 1),
          columns=np.arange(0, Y_numeros.shape[1]))


In [ ]:
X_numeros

### Pasamos de One Hot Encoding a Normal

In [ ]:
# Cambiamos las salidas que están en one hot encoding
# Sumamos 26 ya que luego va juntar los dos dataset. El número del 0 al 9 serán los números del 26 al 35
Y_numeros['Clase'] = Y_numeros.apply(lambda row: np.argmax(row.values) + 26, axis = 1)

# Eliminamos las columnas en one hot encoding
Y_numeros = Y_numeros.drop([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], axis = 1)
# Renombramos la clase para poder concatenarla posteriormente.
Y_numeros = Y_numeros.rename(columns={'Clase': 0})
Y_numeros

### Concatenación de train y test de letras

In [ ]:
# Concatenamos los dataframes de las letras (test y train)
dataframes = [df_letras_train, df_letras_test]

# Ponemos ignore_index para que se vuelva a hacer el index desde cero.
dataset_letras = pd.concat(dataframes, ignore_index = True)
dataset_letras

### Separación de letras en X e Y

In [ ]:
# Dividimos el dataset de las letras en Y y X (labels y data)
Y_letras = dataset_letras['label']
#Y_letras = Y_letras.rename(columns={"label": "Clase"})
X_letras = dataset_letras.drop('label', axis=1)

In [ ]:
Y_letras

### Resize de las imagenes de Numeros a 24x24

In [ ]:
imagenesNumeros = X_numeros.to_numpy().reshape(-1, 64, 64)

In [ ]:
### import numpy as np
import matplotlib.pyplot as plt
import cv2

newX = []
# Hacemos reshape de todas las imagenes
for img in imagenesNumeros:
    newX.append(cv2.resize(img, dsize=(28, 28), interpolation = cv2.INTER_CUBIC))

# Hacemos reshape de las imágenes de nuevo para poder hacer el dataframe
X_numeros = np.asarray(newX).reshape(-1, 784)

In [ ]:
# Volvemos a transformar X_numeros en un dataframe para concatenarlo
X_numeros = pd.DataFrame(data = X_numeros,
          index=np.arange(1, X_numeros.shape[0] + 1),
          columns=np.arange(1, X_numeros.shape[1] + 1))

In [ ]:
X_numeros

In [ ]:
# Normalizamos el dataframe de letras, ya que el dataframe de números 
# estaba normalizado y lo vamos a necesitar para entrenar a la red
# de neuronas correctamente.
import pandas as pd
from sklearn import preprocessing

x = X_letras.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
# Debemos pasarle el nombre de las nuevas columnas ya que si no
# no se concatenan correctamente.
X_letras = pd.DataFrame(x_scaled, columns=list(range(1,785)))

In [ ]:
X_letras

In [ ]:
import matplotlib.pyplot as plt
# Hacemos reshape para que pasen de tener una sola dimensión, a tener 2 (de 784 a 28x28).
# Esto es necesario para imprimir la imagen.
prueba2 = X_letras.to_numpy().reshape(-1, 28, 28)
print(prueba2.shape)
plt.imshow(prueba2[1000], cmap="gray")

In [ ]:
import matplotlib.pyplot as plt
prueba2 = X_numeros.to_numpy().reshape(-1, 28, 28)
print(prueba2.shape)
plt.imshow(prueba2[750], cmap="gray")

## Juntamos los dos dataset y randomizamos las instancias

In [ ]:
X_datasetTotal = [X_letras, X_numeros]
Y_datasetTotal = [Y_letras, Y_numeros]

X_datasetTotal = pd.concat(X_datasetTotal, ignore_index = True)
Y_datasetTotal = pd.concat(Y_datasetTotal, ignore_index = True)
Y_datasetTotal = Y_datasetTotal.rename(columns={0: 'Clase'})
Y_datasetTotal

In [ ]:
# Cada vez que se ejecute esta celda, se obtendrá una aleatorización distinta de los datos.

# Concatenamos las columnas para poder manejarlas mejor al hacer shuffle
result = pd.concat([X_datasetTotal, Y_datasetTotal], axis = 1)

from sklearn.utils import shuffle
# Aleatorizamos las imagenes
result = shuffle(result)

# Volvemos a generar los conjuntos X e Y
X_datasetTotal = result.drop('Clase', axis = 1)
Y_datasetTotal = result['Clase']

X_datasetTotal


## Preparamos las imágenes y las pasamos a One Hot Encoding

In [ ]:
# Hacemos el reshape para que las imagenes tengan el tamaño correcto
X_datasetTotal = X_datasetTotal.values.reshape(X_datasetTotal.shape[0], 28, 28, 1)

from keras.utils.np_utils import to_categorical
Y_datasetTotal = to_categorical(Y_datasetTotal.values)

In [ ]:
Y_datasetTotal.shape

Hay un total de 36 columnas por los siguientes motivos:

- En total, el alfabeto tiene 26 letras. Hay dos que no podemos clasificar: la j y la z. La z no supone un problema ya que se encuentra al final, por lo que directamente no se tiene en cuenta. La j, al estar en el medio, el dataset se salta la etiqueta que debería tener. Esto hace que aun teniendo 24 etiquetas posibles, el array generado al hacer One Hot Encoding, llegue hasta la posición 25.


- Al elegir las nuevas etiquetas de los números, le sumamos 26 debido a que la última etiqueta de las letras es la 25 (por los motivos recién mencionados). Esto hace que 26 + 10 = 36 que es el número de posibles valores que tenemos.

## Separamos en train y test

In [ ]:
import math

nInstancias = math.ceil(0.7 * X_datasetTotal.shape[0])

X_train = X_datasetTotal[:nInstancias]
X_test = X_datasetTotal[nInstancias + 1:]

Y_train = Y_datasetTotal[:nInstancias]
Y_test = Y_datasetTotal[nInstancias + 1:]

In [ ]:
Y_train.shape

## Entrenamiento

In [ ]:
from keras import models
from keras import layers

# Modelo de red neuronal
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(36, activation='softmax'))
          
model.summary()

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

train_generator = train_datagen.flow(X_train, Y_train, batch_size=32)
test_generator = test_datagen.flow(X_test, Y_test, batch_size=32)

from keras import optimizers

model.compile(loss='categorical_crossentropy', 
              optimizer=optimizers.RMSprop(lr=1e-4), 
              metrics=['categorical_crossentropy'])


In [ ]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=X_train.shape[0]//32,
    epochs=10,
    validation_data=test_generator,
    validation_steps=X_test.shape[0]//32
    )


In [ ]:
model.save("modelo.h5")

## Cosas antiguas

In [ ]:
# Hacemos el reshape para que las imagenes tengan el tamaño correcto
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)


# Pasamos las salidas a one hot encoding. Aunque tenemos 24 clases
# obtenemos un array de 25 posiciones. Esto se debe a que la J (numero
# 9) también se tiene en cuenta, aunque no aparezca en ningún momento.
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)


In [ ]:
# import matplotlib.pyplot as plt
# print(X_train.shape)
# plt.imshow(X_train[0][:][:], cmap='gray')
# plt.show()

In [ ]:
# np.save(path_npy + "y_mnist_train.npy", y)
# np.save(path_npy + "X_mnist_train.npy", X)

In [ ]:
print(y_train.shape)
# y = keras.utils.to_categorical(y)
# y

In [ ]:
from keras import models
from keras import layers

# Modelo de red neuronal
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(25, activation='softmax'))
          
model.summary()

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow(X_train, y_train, batch_size=32)
test_generator = test_datagen.flow(X_test, y_test, batch_size=32)

from keras import optimizers

model.compile(loss='categorical_crossentropy', 
              optimizer=optimizers.RMSprop(lr=1e-4), 
              metrics=['categorical_crossentropy'])



In [ ]:
import keras

callbacks = [
    keras.callbacks.TensorBoard(
        log_dir='log/',
    )
]

In [ ]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=X_train.shape[0]//32,
    epochs=10,
    validation_data=test_generator,
    validation_steps=X_test.shape[0]//32,
    callbacks=callbacks
    )

# model.save("modelo.h5")